# Ferrofluid - Part III 

## Table of Contents
1. [Susceptibility with fluctuation formulas](#Susceptibility-with-fluctuation-formulas)
    1. [Derivation of the fluctuation formular](#Derivation-of-the-fluctuation-formular)
    2. [Simulation](#Simulation)
2. [Magnetization curve of a 3D system](#Magnetization-curve-of-a-3D-system)

**Remark**: The equilibration and sampling times used in this tutorial would be not sufficient for scientific  purposes, but they are long enough to get at least a qualitative insight of the behaviour of ferrofluids. They have been shortened so we achieve reasonable computation times for the purpose of a tutorial.

## Susceptibility with fluctuation formulas

In this part we want to calculate estimators for the initial susceptibility, i.e. the susceptibility at zero external magnetic field. One could carry out several simulations with different external magnetic field strenghts and get the initial susceptibility by fitting a line to the results. We want to go a more elegant way by using fluctuation formulas known from statistical mechanics.
In three dimensions the initial susceptibility $\chi_{init}$ can be calculated with zero field simulations through

\begin{equation}
    \chi_{init} = \frac{V \cdot \mu_0}{3 \cdot k_B T} \left( \langle \boldsymbol{M}^2 \rangle - \langle \boldsymbol{M} \rangle^2 \right) = \frac{\mu_0}{3 \cdot k_B T \cdot V} \left( \langle \boldsymbol{\mu}^2 \rangle - \langle \boldsymbol{\mu} \rangle^2 \right)
\end{equation}

where $\boldsymbol{M}$ is the magnetization vector and $\boldsymbol{\mu}$ is the total magnetic dipole moment of the system. In direction $i$ it reads

\begin{equation}
    M_i = \frac{1}{V} \Bigg\langle \sum_{j=1}^N \tilde{\mu}_j^i \Bigg\rangle
\end{equation}

where $\tilde{\mu}_j^i$ is the $j$ th dipole moment in direction $i$.

### Derivation of the fluctuation formular

We want to derive the fluctuation formula. We start with the definition of the magnetic susceptibility. In general this reads

\begin{equation}
    \chi \equiv \frac{\partial}{\partial H} \langle M_{\boldsymbol{H}} \rangle
\end{equation}

with $\langle M_{\boldsymbol{H}} \rangle$ the ensemble averaged magnetization in direction of a homogeneous external magnetic field $\boldsymbol{H}$.

In thermal equilibrium the ensemble average of the magnetization reads

\begin{equation}
    \langle M_{\boldsymbol{H}} \rangle = \frac{1}{V Z_c} \left \lbrack \sum_{\alpha} \mu_{\boldsymbol{H},\alpha} e^{ -\beta E_{\alpha}(H=0) + \beta\mu_0\mu_{\boldsymbol{H},\alpha}H }\right \rbrack
\end{equation}

with $Z_c$ the canonical partition function, $E_{\alpha}(H=0)$ the energy without an external magnetic field $\boldsymbol{H}$, $\beta$ the inverse thermal energy $\frac{1}{k_BT}$, $\mu_{\boldsymbol{H},\alpha}$ the total magnetic dipole moment of the system in direction of the external magnetic field $\boldsymbol{H}$ in microstate $\alpha$ and $V$ the system volume.

Now we insert the magnetization $\langle M_{\boldsymbol{H}} \rangle$ in the definition of the magnetic susceptibility $\chi$ and let the derivative operate on the ensemble average. We get the fluctuation formular

\begin{equation}
    \chi = \frac{\beta\mu_0}{V} \left \lbrack \frac{1}{Z_c}\sum_{\alpha} \mu_{\alpha}^2~ e^{ -\beta E_{\alpha}(H=0) + \beta\mu_0\mu_{\boldsymbol{H},\alpha}H } - \frac{1}{Z_c}\sum_{\alpha} \mu_{\alpha}~ e^{ -\beta E_{\alpha}(H=0) + \beta\mu_0\mu_{\boldsymbol{H},\alpha}H }~~ \frac{1}{Z_c}\sum_{\alpha'}\mu_{\alpha'}~ e^{ -\beta E_{\alpha'}(H=0) + \beta\mu_0\mu_{\boldsymbol{H},\alpha}H }\right \rbrack = \frac{\beta\mu_0}{V} \left \lbrack \langle \mu_{\boldsymbol{H}}^2 \rangle - \langle \mu_{\boldsymbol{H}} \rangle^2 \right \rbrack = \frac{\beta\mu_0}{V} \left(\Delta \mu_{\boldsymbol{H}}\right)^2
\end{equation}

At zero external magentic field ($H = 0$) there is no distinct direction for the system, so we can take the fluctuations $\Delta \mu$ in all directions and divide it by the dimension. Thus we can use more data points of our simulation for the average and get a more precise estimator for the susceptibility. Thus finally the fluctuation formula for the initial susceptibility in three dimensions reads

\begin{equation}
    \chi_{init} = \frac{\beta\mu_0}{3V} \left \lbrack \langle \boldsymbol{\mu}^2 \rangle - \langle \boldsymbol{\mu} \rangle^2 \right \rbrack = \frac{V\beta\mu_0}{3} \left \lbrack \langle \boldsymbol{M}^2 \rangle - \langle \boldsymbol{M} \rangle^2 \right \rbrack
\end{equation}

where $\boldsymbol{\mu}$ and $\boldsymbol{M}$ are defined above.

### Simulation

In this part we want to consider a three dimensional ferrofluid system and compare our result for the initial susceptibility $\chi_{init}$ with them of Ref. <a href='#[1]'>[1]</a>.

First we import all necessary packages

In [1]:
from __future__ import print_function

import espressomd
from espressomd.magnetostatics import DipolarP3M
import numpy as np

Now we setting up all necessary simulation parameters

In [2]:
lj_sigma=1
lj_epsilon=1
lj_cut = 2**(1./6.) * lj_sigma

# magnetic field constant
mu_0 = 1.

# Particles
N = 1000

# Volume fraction
# phi = rho * 4. / 3. * np.pi * ( lj_sigma / 2 )**3.
phi = 0.0262

# Dipolar interaction parameter lambda = mu_0 m^2 /(4 pi sigma^3 kT)
dip_lambda = 3.

# Temperature
kT =1.0

# Friction coefficient
gamma=1.0

# Time step
dt =0.02

# box size 3d
box_size = (N * np.pi * 4./3. * (lj_sigma / 2.)**3. / phi)**(1./3.)

and the system, where we, as we did in **part I**, only commit the orientation of the dipole moment to the particles and take the magnitude into account in the prefactor of Dipolar P3M (for more details see **part I**). 

**Hint:**
It should be noted that we seed both the Langevin thermostat and the random number generator of numpy. Latter means that the initial configuration of our system is the same every time this script is executed. As the time evolution of the system depends not solely on the Langevin thermostat but also on the numeric accuracy and DP3M (the tuned parameters are slightly different every time) it is only partly predefined. You can change the seeds to simulate with a different initial configuration and a guaranteed different time evolution.

In [3]:
system = espressomd.System(box_l=(box_size,box_size,box_size)) 
system.time_step = dt
system.thermostat.set_langevin(kT=kT, gamma=gamma, seed=1)

# Lennard Jones interaction
system.non_bonded_inter[0,0].lennard_jones.set_params(epsilon=lj_epsilon,sigma=lj_sigma,cutoff=lj_cut, shift="auto")

# Random dipole moments
np.random.seed(seed = 1)
dip_phi = np.random.random((N,1)) *2. * np.pi
dip_cos_theta = 2*np.random.random((N,1)) -1
dip_sin_theta = np.sin(np.arccos(dip_cos_theta))
dip = np.hstack((
   dip_sin_theta *np.sin(dip_phi),
   dip_sin_theta *np.cos(dip_phi),
   dip_cos_theta))

# Random positions in system volume
pos = box_size * np.random.random((N,3))

# Add particles
system.part.add(pos=pos, rotation=N*[(1,1,1)], dip=dip)

# Remove overlap between particles by means of the steepest descent method
system.integrator.set_steepest_descent(
    f_max=0,gamma=0.1,max_displacement=0.05)

while system.analysis.energy()["total"] > 5*kT*N:
    system.integrator.run(20)

# Switch to velocity Verlet integrator
system.integrator.set_vv()

# tune verlet list skin
system.cell_system.tune_skin(min_skin=0.4, max_skin=2., tol=0.2, int_steps=100)

# Setup dipolar P3M
system.actors.add(DipolarP3M(accuracy=5E-4,prefactor=dip_lambda*lj_sigma**3*kT))

# tune verlet list skin again
system.cell_system.tune_skin(min_skin=0.4, max_skin=2., tol=0.2, int_steps=100)

# print skin value
print('tuned skin = {}'.format(system.cell_system.skin))

Dipolar P3M tune parameters: Accuracy goal = 5.00000e-04 prefactor = 3.00000e+00
System: box_l = 2.71372e+01 # charged part = 1000 Sum[q_i^2] = 1.00000e+03
Dmesh cao Dr_cut_iL   Dalpha_L     Derr         Drs_err    Dks_err    time [ms]
8    3   4.24763e-01 5.15922e+00 4.95680e-04 3.535e-04 3.474e-04 39      
8    2   4.46568e-01 4.80719e+00 7.01409e-04 3.535e-04 6.058e-04 accuracy not achieved
8    4   3.99469e-01 5.61787e+00 4.94923e-04 3.535e-04 3.464e-04 27      
8    5   3.85514e-01 5.89782e+00 4.97012e-04 3.536e-04 3.493e-04 29      
8    6   3.79408e-01 6.02716e+00 4.95604e-04 3.535e-04 3.473e-04 29      
10   4   3.58118e-01 6.51434e+00 4.98491e-04 3.535e-04 3.514e-04 21      
10   3   3.83865e-01 5.93237e+00 4.99439e-04 3.535e-04 3.528e-04 24      
10   5   3.44854e-01 6.84987e+00 4.94148e-04 3.535e-04 3.452e-04 24      
12   4   3.25943e-01 7.37826e+00 4.99478e-04 3.535e-04 3.528e-04 23      
12   3   3.53222e-01 6.63507e+00 4.98458e-04 3.536e-04 3.514e-04 26      
12   5   3.

Now we equilibrate for a while

In [4]:
print("Equilibration...")
equil_rounds = 10
equil_steps = 1000
for i in range(equil_rounds):
  system.integrator.run(equil_steps)
  print("Dipolar energy: %.2f" % system.analysis.energy()["dipolar"])
  print("progress: {:6.2f} %".format((float(i)+1)/equil_rounds * 100))
print("Equilibration done")

Equilibration...
Dipolar energy: -1037.42
progress:  10.00 %
Dipolar energy: -1152.67
progress:  20.00 %
Dipolar energy: -1064.69
progress:  30.00 %
Dipolar energy: -1056.37
progress:  40.00 %
Dipolar energy: -1079.18
progress:  50.00 %
Dipolar energy: -1204.21
progress:  60.00 %
Dipolar energy: -1141.61
progress:  70.00 %
Dipolar energy: -1211.50
progress:  80.00 %
Dipolar energy: -1128.52
progress:  90.00 %
Dipolar energy: -1093.18
progress: 100.00 %
Equilibration done


As we need the magnetization of our system, we import <tt>MagneticDipoleMoment</tt> from <tt>observables</tt> which returns us the total dipole moment of the system which is the magnetization times the volume of the system.

In [6]:
from espressomd.observables import MagneticDipoleMoment
dipm_tot_calc = MagneticDipoleMoment(ids=system.part[:].id)

Now we set the desired number of loops for the sampling

In [7]:
# Sampling
loops = 2000

and sample the first and second moment of the magnetization or total dipole moment, by averaging over all total dipole moments occuring during the simulation

In [8]:
print('Sampling ...')
# calculate initial total dipole moment
dipm_tot_temp = dipm_tot_calc.calculate()

# initiate variables for sum of first and second moment of total dipole moment
dipm_tot_sum = dipm_tot_temp
dipm_tot_2_sum = np.square(dipm_tot_temp)

# sample dipole moment
for i in range(loops):
    system.integrator.run(10)
    dipm_tot_temp = dipm_tot_calc.calculate()
    
    # add current dipole moment
    dipm_tot_sum = np.sum((dipm_tot_sum, dipm_tot_temp), axis=0)
    dipm_tot_2_sum = np.sum((dipm_tot_2_sum, np.square(dipm_tot_temp)), axis=0)
    
    # print progress only every 10th cycle
    if (i+1)%10 == 0:
        print("progress: {:6.2f} %".format((float(i)+1)/loops * 100))
    
# calculate average first and second moment of total dipole moment
dipm_tot = dipm_tot_sum / loops
dipm_tot_2 = dipm_tot_2_sum / loops
print("Sampling done")

Sampling ...
total dipole moment: [11.203381425565528, 5.14999138751182, -32.03817520059046]
total dipole moment: [8.522210925717882, 4.160214235154754, -21.351373169581457]
total dipole moment: [11.891906836447268, 1.9932921045922414, -11.186391815174552]
total dipole moment: [23.591416461926798, 0.0753408870538606, -17.018930878091727]
total dipole moment: [22.605253302269645, 2.566349421923679, -21.634212949373246]
total dipole moment: [22.24466254865622, 10.577550720290068, -12.322222046818556]
total dipole moment: [9.03984321780536, 12.592815713626424, 0.7852722646581839]
total dipole moment: [1.1901421750846377, 6.241537538729882, 8.335474892519326]
total dipole moment: [-0.4297097702732064, 9.19856829229534, 7.461252926294806]
total dipole moment: [9.668891270920806, 12.995881212311598, 6.4235906447675175]
progress:   0.50 %
total dipole moment: [17.891792399564565, 15.813360474466023, 8.218468062496934]
total dipole moment: [29.12628879791815, 4.888627287888622, 16.756769969315

total dipole moment: [27.102288425417477, 27.10042819575162, -31.136902985773588]
progress:   5.00 %
total dipole moment: [5.1418037725732475, 33.73233720863331, -30.633480103996316]
total dipole moment: [-5.911521905250025, 22.524213802801984, -29.656263281234228]
total dipole moment: [-12.26768486537094, 8.328110745887193, -20.032020525509605]
total dipole moment: [-13.009049704470975, 5.762184501976579, -16.683577716876684]
total dipole moment: [-12.383701500266488, 5.787308234138892, -16.540403956998354]
total dipole moment: [0.1744878362285529, 1.7153818324743417, -18.48428297810659]
total dipole moment: [16.56213605602449, -1.3682380454240222, -19.117757731902447]
total dipole moment: [23.904503009202806, -4.375351041463805, 0.08231179843181077]
total dipole moment: [12.899513479606734, 7.269484751828972, 13.804358652476312]
total dipole moment: [6.896471537588248, 25.009628246822253, 6.258889054355569]
progress:   5.50 %
total dipole moment: [-0.4829739319474138, 16.589336283341

total dipole moment: [18.972678578548386, 23.5612711514211, -13.026675869985397]
total dipole moment: [30.49640411312483, 8.034919390133, 3.482656308597135]
total dipole moment: [37.57117315619294, 8.938546060568639, 4.003374127899291]
progress:  10.00 %
total dipole moment: [46.320419672565016, 21.848507252517187, 1.1146175606928037]
total dipole moment: [46.5323607191145, 27.677857078462278, -4.278111751504132]
total dipole moment: [28.450738459578133, 22.52403330300496, 9.145555632858239]
total dipole moment: [6.688784825320126, 9.48298854141804, 26.706970171151728]
total dipole moment: [8.14438399670134, -4.036259433188522, 25.415352463084556]
total dipole moment: [8.542098430330382, -16.03956691960385, 21.41553903391822]
total dipole moment: [19.496590599860372, -25.06348497124371, 8.732562968451722]
total dipole moment: [25.212322112831618, -25.141192543152307, 1.8441567290287484]
total dipole moment: [17.566318667658916, -25.30920566576268, -0.8405566284948338]
total dipole mome

total dipole moment: [-20.28446280805735, -10.307750049121996, -37.53629488403257]
total dipole moment: [-9.86128800391297, -7.5242124034706475, -35.997204237228146]
total dipole moment: [-2.6938459348246067, -1.618246360386824, -47.61098914734629]
total dipole moment: [10.55165560040023, -7.321167901098578, -46.52844380729643]
total dipole moment: [2.8687681677995713, -4.726159744450632, -27.877849531022434]
progress:  15.00 %
total dipole moment: [-7.937683505683256, -8.894987086411598, -14.280774581795983]
total dipole moment: [-20.49220690225886, -6.074664237310891, -10.66085052590542]
total dipole moment: [-28.973833600476745, -8.186089939428904, -6.163172781708546]
total dipole moment: [-41.19267146335502, -19.89216606522313, -12.22980264704286]
total dipole moment: [-46.33867240808015, -17.459705797455886, -28.893538065465606]
total dipole moment: [-40.20023147957782, -14.637873753979072, -45.15931259918129]
total dipole moment: [-36.78525855063358, -8.938407096089698, -40.56281

total dipole moment: [14.935336519817143, 27.38029023503333, -22.486187146338935]
total dipole moment: [4.366237104102397, 20.22686219444758, -18.957024763697785]
total dipole moment: [22.165955339233985, -0.08437510560346856, -29.344052718361272]
total dipole moment: [46.55362021136077, -6.081603965321646, -36.26412239421504]
total dipole moment: [32.58313578163109, -6.459547702394368, -32.09241175739118]
total dipole moment: [2.6582128137849237, -22.881386424263834, -21.427027140821647]
total dipole moment: [-9.511591075778073, -36.4981414126729, -19.366286197025367]
progress:  20.00 %
total dipole moment: [-5.519807305752474, -36.42203199039799, -9.038792570883794]
total dipole moment: [8.96762988359229, -20.58505322391953, 3.512224076609577]
total dipole moment: [14.480191493609714, -1.4748447722223728, 4.987742439427856]
total dipole moment: [18.446753547629463, 8.435245077821307, 6.258060389794315]
total dipole moment: [20.116934395386657, 7.593517250508216, -11.31837050382927]
t

...

total dipole moment: [-18.469955775444635, -0.4285906023273637, -28.88360440242558]
total dipole moment: [-5.088663627096317, 5.915202596123926, -26.731208668124015]
total dipole moment: [3.46485747094794, 16.837044067416667, -22.181692105352315]
total dipole moment: [9.4330732009283, 22.894736146424407, -5.592847723852335]
total dipole moment: [20.88247699076971, 4.929992697201204, -1.6192274258835195]
total dipole moment: [21.155708088199802, -7.638915566903504, -16.70467735554745]
total dipole moment: [29.446133696431072, -20.28237124470894, -24.31827929000623]
total dipole moment: [28.035759215055496, -2.5362463882403574, -22.676118093552486]
progress:  79.00 %
total dipole moment: [31.676349277229544, 26.841119777897912, -11.310203401292728]
total dipole moment: [33.405954564137424, 37.058056966709394, 11.206817653052347]
total dipole moment: [38.43947433839804, 39.14282313818025, 6.180785144990054]
total dipole moment: [37.63421197295017, 29.816719196227993, -9.097365910232304]
t

total dipole moment: [8.667181416776288, 11.27161839632388, -16.92557552401141]
total dipole moment: [9.234492270548191, 21.408092179235553, -17.051309295430617]
total dipole moment: [6.624866271035847, 27.567543565695082, -5.408825898683426]
total dipole moment: [13.631680244177291, 29.82255329865189, -3.204887714716924]
total dipole moment: [10.724125192546918, 28.458742887848743, -4.880111740509611]
total dipole moment: [6.612388967035022, 16.343352730006846, -6.8862257586812925]
total dipole moment: [10.621224510668185, 11.880859136126707, -1.7921607851038788]
total dipole moment: [20.501786012741075, 18.337432013303196, -4.567971087667481]
total dipole moment: [29.146418600466532, 10.772043029875004, -15.812040829562836]
total dipole moment: [34.91247413220442, 1.8449564188710899, -16.37951128012717]
progress:  84.00 %
total dipole moment: [36.08894467641984, -1.1377512183098424, -25.418544395289995]
total dipole moment: [29.450796190877362, -4.754192059440033, -23.78015045542316]

total dipole moment: [-16.89183514766665, 3.4852649116074788, 41.16192926787978]
progress:  88.50 %
total dipole moment: [-13.352826049200065, 2.1515257226817077, 18.72796313947004]
total dipole moment: [-8.544142598863356, 7.5514349493649116, -2.80570039241406]
total dipole moment: [-13.66284029901244, 19.40928634089483, -13.55509520205364]
total dipole moment: [-11.488646409899843, 19.298547696856936, -15.979306692576827]
total dipole moment: [-5.558856326656927, 5.553402128320805, -21.376513272071413]
total dipole moment: [0.750194813390264, 0.6713414790189182, -23.049322260788877]
total dipole moment: [14.191668533187872, 2.6522137385884608, -28.546650622393273]
total dipole moment: [24.590937624303145, -2.1359820667509517, -36.52992965732923]
total dipole moment: [21.265585133620924, -8.32487211632789, -26.104460536934955]
total dipole moment: [8.500525514778035, -11.5211530470294, -10.474631874712836]
progress:  89.00 %
total dipole moment: [5.220654350607702, -17.274724078371992

total dipole moment: [5.255788381929352, 19.94395075883519, 15.8568723934862]
total dipole moment: [11.574472035325853, 18.9957605258705, 22.8759790884386]
total dipole moment: [13.855859999325808, 20.51806661948296, 19.48876435198514]
progress:  93.50 %
total dipole moment: [18.589822852078065, 26.5689513586831, 17.487873053603067]
total dipole moment: [19.81534801075167, 23.378635112663115, 21.4256638013426]
total dipole moment: [18.080850693526465, 13.079297608000726, 24.29705825368977]
total dipole moment: [15.494716431682413, 13.567250363697976, 14.585091820509966]
total dipole moment: [9.212113646830604, 2.987856868608709, 5.909201278732245]
total dipole moment: [0.1972703371957203, -12.307161843277406, 5.451061934232584]
total dipole moment: [-11.034342600317242, -8.345463254137869, 11.406547278001751]
total dipole moment: [-9.444605558639452, 9.422522726458302, 12.536671563011005]
total dipole moment: [0.6335020344223836, 16.410846186328953, 5.675434887314561]
total dipole mome

total dipole moment: [7.7693836427101095, 6.166230257879941, -23.415959926940186]
total dipole moment: [19.0034072141778, 6.712170992761749, -13.007536499879244]
total dipole moment: [28.396294094999163, -7.431064062678727, -16.335807961522438]
total dipole moment: [27.402294742879928, -26.526305746073838, -12.538826309561825]
total dipole moment: [6.196555336418937, -35.54664198698903, -11.19741525992653]
progress:  98.50 %
total dipole moment: [-11.01492222813486, -32.99542820680397, -6.275530884909119]
total dipole moment: [-13.233669673140673, -24.45005634048929, -4.15874409718782]
total dipole moment: [-8.635437525458206, -26.994939327891313, 4.2508912926399685]
total dipole moment: [-16.92736500157342, -38.94667925012828, 19.219162211334506]
total dipole moment: [-27.551375804616555, -48.896880159604315, 25.824295053292097]
total dipole moment: [-34.017874009991885, -37.838571424131885, 14.34576682422323]
total dipole moment: [-44.28191761703509, -20.7924246529419, -8.12129368577

For the estimator of the initial susceptibility $\chi_{init}$ we need the magnitude of one single dipole moment

In [5]:
# dipole moment
dipm = np.sqrt(dip_lambda*4*np.pi*lj_sigma**3.*kT / mu_0)
print("dipm = {}".format(dipm))

dipm = 6.13996024768


Now we can calculate $\chi_{init}$ from our simulation data

In [10]:
# susceptibility in 3d system
chi = mu_0 /(system.volume() * 3. * kT) * ( np.sum(dipm_tot_2 * dipm**2.) - np.sum(np.square(dipm_tot * dipm)) )

and print the result

In [11]:
print('chi = %.4f' % chi)

chi = 0.8804


Compared with the value $\chi = 0.822 \pm 0.017$ of Ref. <a href='#[1]'>[1]</a> (see table 1) it should be very similar.

Now we want to compare the result with the theoretical expectations.
At first with the simple Langevin susceptibility

In [12]:
chi_L = 8. * dip_lambda * phi
print('chi_L = %.4f' % chi_L)

chi_L = 0.6288


and at second with the more advanced one (see Ref. <a href='#[1]'>[1]</a> eq. (6)) which has a cubic accuracy in $\chi_L$ and reads

\begin{equation}
    \chi = \chi_L \left( 1 + \frac{\chi_L}{3} + \frac{\chi_L^2}{144} \right)
\end{equation}



In [13]:
chi_I = chi_L * ( 1 + chi_L / 3. + chi_L**2. / 144. )
print('chi_I = %.4f' % chi_I)

chi_I = 0.7623


Both of them should be smaller than our result, but the second one should be closer to our one. The deviation of the theoretical results to our simulation result can be explained by the fact that in the Langevin model there are no interactions between the particles incorporated at all and the more advanced (mean-field-type) one of Ref. <a href='#[1]'>[1]</a> do not take occurring cluster formations into account but assumes a homogeneous distribution of the particles. For higher values of the volume fraction $\phi$ and the dipolar interaction parameter $\lambda$ the deviations will increase as the cluster formation will become more pronounced. 

## Magnetization curve of a 3D system

At the end of this tutorial we now want to sample the magnetization curve of a three dimensional system and compare the results with analytical solutions. Again we will compare with the Langevin function but also with the approximation of Ref. <a href='#[2]'>[2]</a> (see also Ref. <a href='#[1]'>[1]</a> for the right coefficients) which takes the dipole-dipole interaction into account. For this approximation, which is a modified mean-field theory based on the pair correlation function, the Langevin parameter $\alpha$ is replaced by

\begin{equation}
    \alpha' = \alpha + \chi_L~L(\alpha) + \frac{\chi_L^{2}}{16} L(\alpha) \frac{d L(\alpha)}{d\alpha}
\end{equation}

where $\chi_L$ is the Langevin susceptibility

\begin{equation}
   \chi_L = \frac{N}{V}\frac{\mu_0 \mu^2}{3k_BT} = 8 \cdot \lambda \cdot \phi
\end{equation}

Analogous to **part II** we start at zero external magnetic field and increase the external field successively. At every value of the external field we sample the total dipole moment which is propotional to the magnetization as we have a fixed volume.

First we create a list of values of the Langevin parameter $\alpha$. As we already sampled the magnetization at zero external field in the last section, we take this value and continue with the sampling of an external field unequal zero

In [28]:
alphas = [0.25, 0.5, 1, 2, 3, 4, 8]

Now for each value in this list we sample the total dipole moment / magnetization of the system for a while. Have in mind that we have only commited the orientation of the dipole moments to the particles. Thus we have to commit $H\cdot \mu$ as the external magnetic field to **ESPResSo**, where $\mu$ is the magnitude of a single magnetic dipole moment.

As in **part II** we use the same system for every value of the Langevin parameter $\alpha$. Thus we use that the system is already pre-equilibrated from the previous run so we save some equilibration time. For scientific purposes one would use a new system for every value for the Langevin parameter to ensure that the systems are independent and no correlation effects are measured. Also one would perform more than just one simulation for each value of $\alpha$ to increase the precision of the results.

In [14]:
# remove all constraints
system.constraints.clear()

# list of magnetization in field direction
magnetization = []

# append result for alpha=0 from previous chapter
magnetization.append(np.average(dipm_tot))

# number of loops for sampling
loops = 500

for alpha in alphas:
    print("Sample for alpha = {}".format(alpha))
    H_dipm = (alpha*kT)
    H_field = [H_dipm,0,0]
    print("Set magnetic field constraint...")
    H_constraint = espressomd.constraints.HomogeneousMagneticField(H=H_field)
    system.constraints.add(H_constraint)
    print("done")
    
    # Equilibration
    print("Equilibration...")
    for i in range(equil_rounds):
        system.integrator.run(equil_steps)
        print("Dipolar energy: %.2f" % system.analysis.energy()["dipolar"])
        print("progress: {:6.2f} %".format((float(i)+1)/10 * 100))
    print("Equlilibration done \n")
    
    # Sampling
    print("Sampling...")
    magn_temp = 0
    for i in range(loops):
        system.integrator.run(20)
        magn_temp += dipm_tot_calc.calculate()[0]
        print("progress: {:6.2f} %".format((float(i)+1)/loops * 100))
    
    # save average magnetization
    magnetization.append(magn_temp / loops)
    print("Sampling for alpha = {} done \n".format(alpha))
    print("magnetizations = {}".format(magnetization))
    print("total progress: {:6.2f} %\n".format((float(alphas.index(alpha))+1)/len(alphas) * 100))
    
    # remove constraint
    system.constraints.clear()
print("Magnetization curve sampling done")

Now we define the Langevin function and the modified mean-field-approximation of the Langevin parameter of Ref. <a href='#[2]'>[2]</a>

In [15]:
# Langevin function
def L(y):
    return np.cosh(y)/np.sinh(y)-1/y

In [16]:
#  second order mean-field-model from Ref. [2]
def alpha_mean_field(alpha, dip_lambda, phi):
    chi = 8. * dip_lambda * phi
    return alpha + chi * L(alpha) + chi**2. / 16. * L(alpha) * ( 1./(alpha**2.) - 1./((np.sinh(alpha))**2.) )

We also want to plot the linear approximation at $\alpha = 0$ to see for which values of $\alpha$ this approximation holds. We use the initial susceptibility calculated in the first chapter of this part as the gradient. As we want the gradient of $M^*$ with respect to $\alpha$ which fullfulls the relation

\begin{equation}
    \frac{\partial M^*}{\partial \alpha} = \frac{1}{M_{sat}}\frac{\partial M}{\partial \left( \frac{\mu_0\mu}{k_BT} H\right)} = \frac{k_BT~V}{\mu_0\mu^2N}\frac{\partial M}{\partial H} = \frac{k_BT~V}{\mu_0\mu^2N}~\chi
\end{equation}

we have to scale our calculated initial susceptibility $\chi_{init}$ by a factor to get it in our dimensionless units.

Now we plot the resulting curves together with our simulation results and the linear approximation

In [17]:
import matplotlib.pyplot as plt

In [35]:
y = np.arange(0.01,10, 0.1, dtype=float).tolist()

alphas.insert(0, 0.0)


L_func = []
L_mean_field = []
init_susceptibility = []

for i in y:
    L_func.append(L(i))
    L_mean_field.append(L(alpha_mean_field(i, dip_lambda, phi)))
    init_susceptibility.append(i*system.volume()*kT/(N*mu_0*dipm**2)*chi)

# devide all entries in the magnetization list by N to get the dimensionless magnetization
magnetization_star = []
for i in range(len(magnetization)):
    magnetization_star.append(magnetization[i] / N)

print('magnetization_star = {}'.format(magnetization_star))
    
    
plt.figure(figsize=(10,10))
plt.ylim(0, 1.)
plt.xlabel(r'$\alpha$', fontsize=20)
plt.ylabel(r'$M^*$', fontsize=20)
plt.plot(y, L_func, label='Langevin function')
plt.plot(y, L_mean_field, label='modified mean-field-theory')
plt.plot(alphas, magnetization_star, 'o', label='simulation results')
plt.plot(y, init_susceptibility, label=r'linear approximation at $\alpha = 0$')
plt.legend(fontsize=20)
plt.show()

We can see that the magnetization curve where we used the Langevin parameter of the modified mean-field-theory is closer to our simulation results. Also we can clearly see that the linear approximation holds only for very small values of $\alpha$.

At this point is should be mentioned, that the modified mean-field-model assumes a spatially homogeneous system which is not the case at higher volume fractions $\phi$ and dipolar interaction parameters $\lambda$ as the particles form clusters. We can already see this with our simulation results as they visibly deviate from the modified mean-field-model.

At sufficiently high volume fractions $\phi$ and dipolar interaction parameters $\lambda$ these clusters can be thus rigid, that simulation with normal methods are impossible as the relaxation times exceeds normal simulation times by far resulting in strongly correlated configurations and thus measurements.

<a id='[1]'></a>[1] Zuowei Wang, Christian Holm, and Hanns Walter Müller. “Molecular dynamics
study on the equilibrium magnetization properties and structure of ferrofluids”. In:
Phys. Rev. E 66 (2 Aug. 2002), p. 021405. doi: 10.1103/PhysRevE.66.021405.
url: https://link.aps.org/doi/10.1103/PhysRevE.66.021405.

<a id='[2]'></a>[2] Alexey O. Ivanov and Olga B. Kuznetsova. “Magnetic properties of dense ferrofluids:
An influence of interparticle correlations”. In: Phys. Rev. E 64 (4 Sept. 2001),
p. 041405. doi: 10.1103/PhysRevE.64.041405. url: https://link.aps.org/
doi/10.1103/PhysRevE.64.041405.